In [1]:
# %matplotlib ipympl 
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import boxcox
from scipy.stats import yeojohnson

In [2]:
# load all data into dataframe
def load_data(path, file_names, aliases):
    dates = {}
    for data_set_idx in range(len(data_files)):
        cur_alias = aliases[data_set_idx]
        with open(path + data_files[data_set_idx] + '.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            spamreader.__next__()
            for row in spamreader:
                try:
                    cur_date = datetime.datetime.strptime(row[0], '%m/%d/%Y')

                except:
                   continue
                if not cur_date in dates:
                    dates[cur_date] = {}
#                     # need to generalize here
#                 if data_set_idx == 0 or data_set_idx == 2:
#                     dates[cur_date][cur_alias] = float(row[4])
#                 elif data_set_idx == 1 or data_set_idx == 3 or data_set_idx == 4 or data_set_idx == 5:
                try:
                    dates[cur_date][cur_alias] = float(row[1])
                except:
                    print(row[1])
                    print(cur_alias)
                    print(row)
                    

    df = pd.DataFrame.from_dict(dates, orient='index')
    df.columns = aliases
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Date'})
    df = df.sort_values('Date')
    df = df.reset_index(drop=True)
    return df


In [3]:
# not_null = df.query(baseline_asset + ".notnull()")
# not_null.reset_index(drop = True, inplace = True)

In [4]:
# adds correlation metrix to dataframe
def add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            cur_price = row[asset]
            if idx > days_out: # check for enough data
                # get percent change
                last_time_period = df.loc[idx - days_out - 1].at[asset]
                time_period_change = (cur_price - last_time_period)/last_time_period
                cors[alias_idx].append(time_period_change)           
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance].at[asset]):
                fut_val = df.iloc[idx + pred_distance].at[asset]
                time_period_change = (fut_val - cur_price)/cur_price  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [5]:
def add_pred_differences(pred_distance, baseline_asset, assets, df):
    for idx, asset in enumerate(assets):
#       for idx2, asset2 in enumerate(assets[idx + 1: ]): if you want all differences
        if asset != baseline_asset:
            change1 = df[asset + "_fut_" + str(pred_distance) + "dys"]
            change2 = df[baseline_asset + "_fut_" + str(pred_distance) + "dys"]
            diff = change1 - change2 
            df.insert(df.shape[1], asset + "_" + baseline_asset + "_" + str(pred_distance) + "dys_diff", diff, True)

In [6]:
# not_null.iloc[5700:5750,[0, 7,8, 9, 10,11,12,13,14,15,16,17,18,19,20]]

In [7]:
# not_null.iloc[-160:,0: 10]

In [8]:
# not_null.columns

In [9]:
# #use sklearn.preprocessing.PowerTransformer instead

# plt.figure()

# column_name = 'sp_20_dys'
# column = not_null[column_name] 
# column = column[~np.isnan(column)]
# print(column)
# # column += np.array([1 for i in range(len(column))])
# # print(column)
# plt.figure()
# plt.hist(column , color = 'red', bins = 500, density=True)
# mean = np.mean(column)
# std = np.std(column)
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))


# plt.figure()
# plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# mean = np.mean(yeojohnson(column)[0])
# std = np.std(yeojohnson(column)[0])
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])) , color = 'red', bins = 500, density=True)
# # mean = np.mean(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))
# # std = np.std(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))



# # column += np.array([1 for i in range(len(column))])
# # column = np.log(column)
# # plt.figure()
# # plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# # mean = np.mean(yeojohnson(column)[0])
# # std = np.std(yeojohnson(column)[0])
# # print(mean)
# # print(std)
# # x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(sp_not_null['sp_fut_2wks'], color = 'red', bins = 500)
# # plt.hist(sp_not_null['re_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['bnd_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['gld_fut_2wks'], color = 'yellow', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['eu_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['jp_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# plt.show()

In [10]:
# fig = plt.figure()
# ax = fig.add_subplot(projection = '3d')

# ax.scatter(not_null["sp_last_month"], not_null["re_last_month"], not_null["re_sp_2wk_diff"])
# ax.set_xlabel('sp_last_month')
# ax.set_ylabel('re_last_month')
# ax.set_zlabel('re_sp_2wk_diff')
# plt.show()

In [11]:
def get_rvs(baseline, df, aliases, pred_distance, print_mats=False):
    rvs = {}
    
    valid_cols = []
    for col in df.columns:
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            valid_cols.append(col)
    valid_cols.append(None)
    
    for asset in aliases:
        if asset != baseline:
            valid_cols[-1] = (asset + "_" + baseline + "_" + str(pred_distance) + "dys_diff")

            cov_mat = df[valid_cols]
            cov_matrix = pd.DataFrame.cov(cov_mat)
            cov_mat = cov_mat.cov()
            cov_mat = cov_mat.to_numpy()
#             if print_mats:
            print(asset)
            print(cov_matrix)


            # means of values
            means = []
            for col in valid_cols:
                means.append(np.mean(df[col]))
            if print_mats:
                print(means)

            rv = multivariate_normal(mean=means, cov=cov_mat, allow_singular=True)
            rvs[asset] = rv
    return rvs
        


def predict(asset, baseline, rv, inputs, get_plots=False, do_print=False, get_50_pt = False):
    START = -.22
    STOP = .22
    INCREMENT = .00005

    probs = []
   
    x = np.arange(START, STOP, INCREMENT)
    inputs.append(None)
    for val in x:
        # make an array with all the current values
        # insert past month performance
        #"sp", "re", "bnd", "eu", "jp", "gld", future difference
        inputs[-1] = val
        probs.append(rv.pdf(inputs))


    cdf = []
    for idx in range(x.size - 1):
        cur_prob = probs[idx]
        next_prob = probs[idx + 1]
        rieman_sum = min(cur_prob, next_prob) * INCREMENT
        rieman_sum += max(cur_prob, next_prob) - min(cur_prob, next_prob) * INCREMENT / 2
        if len(cdf) > 0:
            cdf.append(rieman_sum + cdf[-1])
        else:
            cdf.append(rieman_sum)
                                 
    if get_plots:
        fig1 = plt.figure()
        ax = fig1.add_subplot(111)
        plt.title("pdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x, probs/cdf[-1])
        plt.show()

    for idx in range(len(cdf)):
        cdf[idx] /= cdf[-1]

    if get_plots:
        fig2 = plt.figure()
        ax = fig2.add_subplot(111)
        plt.title("cdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x[:-1], cdf)
        plt.show() 
        
    if get_50_pt:
        # find 50% point
        cur_prob = 0
        idx = 0
        while(cur_prob < .5):
            cur_prob = cdf[idx]
            idx += 1
        fiftyfiftypt = x[idx]
        if do_print:
            print("50 50 change to be above or below")
            print(x[idx])


    #find expected value
    expected_value = 0
    for idx in range(len(cdf)):
        if idx == 0:
            expected_value += cdf[idx] * x[idx]
        else:
            cur_prob = cdf[idx - 1]
            next_prob = cdf[idx]
            actual_prob = next_prob - cur_prob
            expected_value += actual_prob * x[idx]
                                 
    if do_print:
        print("Expected Value")
        print(expected_value)
                                 
    if get_50_pt:
        return fiftyfiftypt, expected_value
    else:
        return expected_value

In [12]:
def test_preds(assets, baseline, pred_distance, df):
    rvs = get_rvs(baseline, df, assets, pred_distance)
    preds = {}
    actuals = {}
    for asset in assets:
        if asset != baseline:
            preds[asset] = []
            actuals[asset] = []
    
    pred_columns = []
    for col_idx, col in enumerate(df.columns):
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            pred_columns.append(col_idx)
            
            
    print('start')
    for idx, row in df.iterrows():
        for asset in assets:
            if asset != baseline:
#                 print(asset)
                col_name = asset + "_" + baseline  + "_" + str(pred_distance) + "dys_diff"
                actual = row[col_name]
                actuals[asset].append(actual)
#                 print(actual)
                columns = []
                pred_input = df.iloc[idx, pred_columns]
                if not pred_input.isnull().any():
                    prediction = predict(asset, baseline, rvs[asset], pred_input.tolist(), get_plots=False)
                    preds[asset].append(prediction)
#                     print(prediction)
                else:
                    preds[asset].append(None)
#                     print(None)
#                 print("---------")
        if idx % 10== 0:
            print(idx)

    return preds, actuals            

In [18]:
# inputs here

# path = 'C:\\Users\\plant\\OneDrive\\Documents\\ProbStock\\'
path = ''
baseline_asset = 'sp'


file_name_SP = 'SPY'  
# file_name_RE = 'RealEstateIndexDailySince2013'
file_name_RE = 'IYR'
file_name_BND = 'isharesBondIndexSince2003'
file_name_EU = 'USD_EURHistoricalData'
file_name_JPY = 'USD_JPYHistoricalData'
file_name_GOLD = 'GoldFuturesHistoricalData'
file_name_RUT = 'IWM'

# data_files = [file_name_SP, file_name_RE, file_name_BND, file_name_EU, file_name_JPY, file_name_GOLD, file_name_RUT]
# aliases = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']

# # input correlaries
# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut']
# cor_days_out = [20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 252, 252, 252, 252, 252, 252, 252, 60, 60, 60, 60, 60, 60, 60]
# pred_distance = 10
# assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']


data_files = [file_name_SP, file_name_RE, file_name_BND, file_name_EU, file_name_JPY, file_name_GOLD, file_name_RUT]
aliases = ["sp", "re", "bnd", "eu", "jp", "gld", "rut"]

# input correlaries
cor_assets = ['sp', "re", "bnd", "eu", "jp", "gld", "rut", "sp", "re", "bnd", "eu", "jp", "gld", "rut", "re", "bnd", "eu", "jp", "gld", "rut", "sp", "re", "bnd", "eu", "jp", "gld", "rut"]
cor_days_out = [20,  20,   20,    20,   20,     20,    20,   10,   10,    10,   10,   10,    10,    10,  252,  252,  252,   252,   252,   252,  60,   60,    60,   60,   60,    60,   60]
pred_distance = 10
assets = ["sp", "re", "bnd", "eu", "jp", "gld", "rut"]

In [19]:
df = load_data(path, data_files, aliases)


FileNotFoundError: [Errno 2] No such file or directory: 'SPY.csv'

In [21]:
df.iloc[0:1]

,Date,sp,re,bnd,eu,jp,gld,rut
0,1987-09-10,NaN,NaN,NaN,168.970001,NaN,NaN,NaN


In [22]:
add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets)
add_pred_differences(pred_distance, baseline_asset, assets, df)

In [23]:
df.shape

(9430, 48)

In [24]:
df.iloc[7550:7551, [0, 1, 6, 7]]

,Date,sp,gld,rut
7550,2017-01-09,2268.9,77.889999,108.470001


In [164]:
# rvs = get_rvs(baseline_asset, df, aliases, pred_distance, print_mats = False)
# predict("bnd", baseline_asset, rvs, [-0.060534,0.015215,-0.030221,0.018300,-0.000830,-0.026808,-0.058842,0.000109,-0.021193,0.033956], get_plots=True)

In [165]:
preds, actuals = test_preds(aliases, baseline_asset, pred_distance, df)

re
                  sp_20_dys  re_20_dys  bnd_20_dys  eu_20_dys  jp_20_dys  \
sp_20_dys          0.002271  -0.000275    0.000143   0.002618   0.000066   
re_20_dys         -0.000275   0.000733    0.000225  -0.000315  -0.000504   
bnd_20_dys         0.000143   0.000225    0.000726   0.000239  -0.000452   
eu_20_dys          0.002618  -0.000315    0.000239   0.003578   0.000013   
jp_20_dys          0.000066  -0.000504   -0.000452   0.000013   0.001940   
gld_20_dys         0.002168  -0.000373    0.000111   0.002822   0.000365   
rut_20_dys         0.000101  -0.000066   -0.000142   0.000080   0.000162   
sp_10_dys          0.001180  -0.000129    0.000084   0.001329  -0.000043   
re_10_dys         -0.000156   0.000381    0.000119  -0.000165  -0.000259   
bnd_10_dys         0.000075   0.000120    0.000380   0.000138  -0.000250   
eu_10_dys          0.001391  -0.000163    0.000122   0.001851  -0.000044   
jp_10_dys          0.000095  -0.000265   -0.000230   0.000042   0.001012   
gld_10_dy

gld
                   sp_20_dys  re_20_dys  bnd_20_dys  eu_20_dys  jp_20_dys  \
sp_20_dys           0.002271  -0.000275    0.000143   0.002618   0.000066   
re_20_dys          -0.000275   0.000733    0.000225  -0.000315  -0.000504   
bnd_20_dys          0.000143   0.000225    0.000726   0.000239  -0.000452   
eu_20_dys           0.002618  -0.000315    0.000239   0.003578   0.000013   
jp_20_dys           0.000066  -0.000504   -0.000452   0.000013   0.001940   
gld_20_dys          0.002168  -0.000373    0.000111   0.002822   0.000365   
rut_20_dys          0.000101  -0.000066   -0.000142   0.000080   0.000162   
sp_10_dys           0.001180  -0.000129    0.000084   0.001329  -0.000043   
re_10_dys          -0.000156   0.000381    0.000119  -0.000165  -0.000259   
bnd_10_dys          0.000075   0.000120    0.000380   0.000138  -0.000250   
eu_10_dys           0.001391  -0.000163    0.000122   0.001851  -0.000044   
jp_10_dys           0.000095  -0.000265   -0.000230   0.000042   0.00101

80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2220
2230
2240
2250
2260

In [166]:
import pickle 

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

with open('actuals.pkl', 'wb') as f:
    pickle.dump(actuals, f)



In [2]:
import pickle
with open("preds.pkl",'rb') as f:
    preds = pickle.load(f)
    
with open("actuals.pkl",'rb') as f:
    actuals = pickle.load(f)

In [5]:
sign_cor_neg = 0
sign_cor_pos = 0
incor_actual_neg = 0
incor_actual_pos = 0
total_diff = 0
total_count = 0
total_correct = 0

for pred, actual in zip(preds['rut'], actuals['rut']):
    if pred != None and not pd.isna(actual):
        if pred < 0 and actual < 0:
            sign_cor_neg += 1
            total_correct += 1
        elif pred > 0 and actual > 0:
            sign_cor_pos += 1
            total_correct += 1
        elif actual < 0:
            incor_actual_neg += 1
        else:
            incor_actual_pos +=1
        total_count += 1
        total_diff += abs(pred - actual)

In [6]:
print("pred neg actual neg")
print(sign_cor_neg)
print("pred pos actual pos")
print(sign_cor_pos)
print("pred pos actual neg")
print(incor_actual_neg) 
print("pred neg actual pos")
print(incor_actual_pos )

print("-----")
print("total neg")
print(sign_cor_neg + incor_actual_neg)
print("total pos")
print(sign_cor_pos + incor_actual_pos)
print("----")
print("ave diff")
print(total_diff/total_count)
print("percent correct")
print(total_correct/total_count)


pred neg actual neg
1544
pred pos actual pos
657
pred pos actual neg
762
pred neg actual pos
849
-----
total neg
2306
total pos
1506
----
ave diff
0.02328688436271421
percent correct
0.5773871983210913


In [64]:
total = 1
period_counts = 0

not_null = df.query(baseline_asset + ".notnull()")

count = 0

for date, pred, actual in zip(df["Date"], preds['rut'], actuals['rut']):
    if pred != None and not pd.isna(actual):
        if count == 10:
            if pred > 0:
                total *= (1 + 10 * pred * actual)
                print("long: " + str(date) + ": " + str(total))
            if pred < 0:
                total *= (1 + 10  * abs(pred) * -(actual))
                print("shorted: " + str(date) + ": " + str(total))
            period_counts += 1
    #         else:
    #             total *= (1 + actual)
            
            count = 0
        count += 1
        
        

print(total)
print(period_counts)

long: 2004-10-04 00:00:00: 1.0002927931013184
shorted: 2004-10-19 00:00:00: 1.000567035948433
long: 2004-11-02 00:00:00: 0.9999427971453418
long: 2004-11-19 00:00:00: 0.9988842454334177
shorted: 2004-12-08 00:00:00: 0.9997283147326922
shorted: 2005-01-05 00:00:00: 0.9996697951715815
shorted: 2005-01-25 00:00:00: 1.0010558413648178
long: 2005-02-14 00:00:00: 1.0009822300068214
long: 2005-03-03 00:00:00: 1.0020023529626585
long: 2005-03-23 00:00:00: 1.0016193220460914
long: 2005-04-13 00:00:00: 1.0018533985062605
shorted: 2005-04-28 00:00:00: 1.0022200710851832
long: 2005-05-12 00:00:00: 1.0016474682659027
long: 2005-06-02 00:00:00: 1.0014131746294654
long: 2005-06-17 00:00:00: 1.0038030311651436
shorted: 2005-07-07 00:00:00: 1.0042359529590101
long: 2005-07-22 00:00:00: 1.004151482076933
long: 2005-08-08 00:00:00: 1.0049412857962836
shorted: 2005-08-24 00:00:00: 1.0066516384333675
shorted: 2005-09-09 00:00:00: 1.006149287589316
long: 2005-09-26 00:00:00: 1.0075678806655732
long: 2005-10